In [9]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import RK45
from numba import njit

#Вычисляем расстояние по координатам кси и эта - это пятое уравнение в системе
@njit(fastmath=True, cache=True)
def r(ksi, eta):
    return np.sqrt(ksi ** 2 + eta ** 2)
    
#Вычисляем вторую производную кси - это третье уравнение в системе
@njit(fastmath=True, cache=True)
def fksi2(ksi, eta, eta1):
    return 2*omega*eta1-(G*m2/((mu**2) * r(ksi, eta) ** 3)+al1)*ksi

#Вычисляем вторую производную этта - это четвертое уравнение в системе
@njit(fastmath=True, cache=True)
def feta2(ksi, eta, ksi1):
    return -2*omega*ksi1-G*m2/((mu**2) * r(ksi, eta) ** 3)*eta

#Интеграл якоби для контроля точности по энергии
@njit(fastmath=True, cache=True)
def eps(ksi, eta, ksi1, eta1):
    return (ksi1 ** 2 + eta1 ** 2) / 2. - G * m2 / (mu ** 2 * r(ksi, eta)) + al1 * ksi ** 2 / 2

#Позже
@njit(fastmath=True, cache=True)
def check(y):
    ksi, eta, ksi1, eta1 = y
    return np.abs(e1 / eps(ksi, eta, ksi1, eta1) - 1)

#Вычисляем значение кси при заданной константе с
@njit(fastmath=True, cache=True)
def ksi0(c):
    return -mod_ksi1t + c

#Вычисляем первую производную эта - начальное значение первой производной (в момент времени 0)
@njit(fastmath=True, cache=True)
def eta10(c1):
    return np.sqrt(2 * (e1 + G * m2 / ((mu ** 2) * np.abs(ksi0(c1)))) - al1 * ksi0(c1) ** 2)

#Собираем все уравнения в кучу. Массив 4*4
@njit(fastmath=True, cache=True)
def foo(t, y):
    # y = [ksi, eta, ksi1, eta1]
    # dy = [ksi1, eta1, ksi2, eta2]
    ksi = y[0]
    eta = y[1]
    ksi1 = y[2]
    eta1 = y[3]
    ksi2 = fksi2(ksi, eta, eta1)
    eta2 = feta2(ksi, eta, ksi1)
    dy = [ksi1, eta1, ksi2, eta2]
    return dy

#Ищем предварительную орбиту
def look_for_orbit(C):
    for c in C:
        ksi_0 = ksi0(c)
        eta_0 = 0
        ksi1_0 = 0
        eta1_0 = eta10(c)
        y0 = [ksi_0, eta_0, ksi1_0, eta1_0]
        y_arr = [y0]
        t_arr = [0]

        # for t in np.arange(0, 300, dt):
        sol = RK45(fun=foo, t0=0, y0=y0, t_bound=500, max_step=dt)
        while sol.status != 'finished':
            try:
                sol.step()
                _t = sol.t
                y = sol.y
                y_arr.append(y)
                t_arr.append(_t)
            except RuntimeError:
                print('beep')
                break
        # y0 = y
        y_arr = np.array(y_arr)
        plt.plot(y_arr[:, 0], y_arr[:, 1], '.', markersize=1.5,
                 label='c=' + str(c) + ' T='+str(t_arr[-1]), color='orangered')
        plt.legend()
        plt.grid()
        plt.savefig(f'c={c}.jpg')
        plt.clf()
        plt.cla()
        plt.close()

#Кажется рисует конечный график
def draw(y_arr, t_arr, c):
    if len(y_arr) > 4000:
        max_points_n = 2000.
        d_arr = int(len(y_arr)/max_points_n)
        y_arr = np.array(y_arr[::d_arr])
        t_arr = np.array(t_arr[::d_arr])
    else:
        y_arr = np.array(y_arr)
        t_arr = np.array(t_arr)

    delta = [check(i) for i in y_arr]
    delta = np.array(delta)
    fig, axs = plt.subplots(2, figsize=(6, 9.5), dpi=125)
    axs[0].set_title('Орбита звезды')
    axs[1].set_title('Точность вычислений от времени')
    axs[0].set(xlabel=r'$\xi$', ylabel=r'$\eta$')
    axs[0].plot(y_arr[:, 0], y_arr[:, 1], 'b.', markersize=3, linewidth=0.5,
                label='c=' + str(np.around(c, 5))+', T='+str(t_arr[-1]))
    axs[1].set(xlabel='t, Myr', ylabel='err')
    axs[1].plot(t_arr, delta, 'r.')
    for ax in fig.get_axes():
        ax.grid()
    axs[0].legend(loc='best')
    fig.savefig(f'orbit_{str(np.around(c, 5))}.jpg')
    #np.savetxt(f'orbit_{str(np.around(c, 5))}.txt', y_arr, fmt='%5f', delimiter='\t')
    #err_out = np.column_stack([delta*1e14, t_arr])
    #np.savetxt(f'err_{str(np.around(c, 5))}_10e-14.txt', err_out, fmt='%5f', delimiter='\t')

#
def search(c0, c1, dc, err):
    if err < 1e-5:                                                          # функция вызывает сама себя, но с другими начальными условиями, это ограничение на ошибку и dc
        err = 1e-5                                                          # чтобы не искать бесконечно решение, там где его нет
    #if dc < 1e-5:
    #    dc = 1e-5
    minr = []
    minc = []
    mint = []
    for c in np.arange(start=c0, stop=c1, step=dc):
    #for c in np.arange(start=c0, stop=c1, step=-dc):
        print(c)
        ksi_0 = ksi0(c)
        eta_0 = 0
        ksi1_0 = 0
        eta1_0 = eta10(c)
        y0 = [ksi_0, eta_0, ksi1_0, eta1_0]
        y_arr = [y0]
        t_arr = [0]
        R=[]
        C=[]
        T=[]
        sol = RK45(fun=foo, t0=0, y0=y0, t_bound=150, max_step=dt)
        i = 0
        while sol.status != 'finished':
            try:
                sol.step()
                i = i + 1
                t = sol.t
                y = sol.y
                y_arr.append(y)
                t_arr.append(t)
                # y = [xi, eta = 0, dxi = 0, deta]
                if i < 5000:                                                     # чтобы не рассматривать точки графика в самом начале счета времени, когда звезда не прошла полный круг
                    continue                                                     # отступ по времени получается dt * i = 0.001 * 5000 = 5 млн лет, а орбитальный период порядка 150 млн лет,
                rr = np.sqrt((y[0] - ksi_0)**2+y[1]**2)
                if rr > err:                                                # если расстояние больше грубой ошибки переходим на следующий шаг по времени
                    continue
                R.append(rr)
                T.append(t)
                C.append(c)
            except RuntimeError:
                print('beep')
                break
        minr.append(min(R))
        print('minr=', min(R))
        minc.append(C[R.index(min(R))])
        print('minc=', C[R.index(min(R))])
        mint.append(T[R.index(min(R))])
        print('mint=',T[R.index(min(R))])
    print(minr)
    print(minc)
    print(mint)
    print(min(minr))
    print(minr.index(min(minr)))
    print(mint[minr.index(min(minr))])
    print(minc[minr.index(min(minr))])
    plt.plot(minc, minr, '.', markersize=3,
             label= 'r=' + str(min(minr)) + 'c=' + str(minc[minr.index(min(minr))]) + ' T=' + str(mint[minr.index(min(minr))]), color='orangered')
    plt.legend()
    plt.ylabel('r')
    plt.xlabel('c')
    plt.grid()
    plt.savefig(f'minr.jpg')
    plt.clf()
    plt.cla()
    plt.close()



# def runge(fksi2, feta2, t0, ksi0, eta0, ksi10, eta10, T0, T, dt):
#     ksi = ksi0
#     eta = eta0
#     ksi1 = ksi10
#     eta1 = eta10
#     t = t0
#     n = int((T - T0) / dt) + 1
#     print('Step:', dt)
#     sol1 = {}
#     sol2 = {}
#     sol3 = {}
#     sol4 = {}
#     for i in range(n):
#         K11 = dt * ksi1
#         K21 = dt * (ksi1 + K11 / 2)
#         K31 = dt * (ksi1 + K21 / 2)
#         K41 = dt * (ksi1 + K31)
#
#         K12 = dt * eta1
#         K22 = dt * (eta1 + K12 / 2)
#         K32 = dt * (eta1 + K22 / 2)
#         K42 = dt * (eta1 + K32)
#
#         K13 = dt * fksi2(ksi, eta, eta1)
#         K23 = dt * fksi2(ksi + K13 / 2, eta + K13 / 2, eta1 + K13 / 2)
#         K33 = dt * fksi2(ksi + K23 / 2, eta + K23 / 2, eta1 + K23 / 2)
#         K43 = dt * fksi2(ksi + K33, eta + K33, eta1 + K33)
#
#         K14 = dt * feta2(ksi, eta, ksi1)
#         K24 = dt * feta2(ksi + K14 / 2, eta + K14 / 2, ksi1 + K14 / 2)
#         K34 = dt * feta2(ksi + K24 / 2, eta + K24 / 2, ksi1 + K24 / 2)
#         K44 = dt * feta2(ksi + K34, eta + K34, ksi1 + K34)
#
#         sol1[t] = ksi
#         sol2[t] = eta
#         sol3[t] = ksi1
#         sol4[t] = eta1
#
#         t = t + dt
#         delta_ksi = (K11 + 2 * K21 + 2 * K31 + K41) / 6
#         ksi = ksi + delta_ksi
#         delta_eta = (K12 + 2 * K22 + 2 * K32 + K42) / 6
#         eta = eta + delta_eta
#         delta_ksi1 = (K13 + 2 * K23 + 2 * K33 + K43) / 6
#         ksi1 = ksi1 + delta_ksi1
#         delta_eta1 = (K14 + 2 * K24 + 2 * K34 + K44) / 6
#         eta1 = eta1 + delta_eta1
#     return sol1, sol2, sol3, sol4

al1 = -0.001976995
G = 0.004535
m1 = 1.
m2 = 499.
mu = 1. + m1 / m2
mod_ksi1t = 10.44666776
omega = 0.02829174
e1t = -3 * G * m2 / (2 * (mu ** 2) * mod_ksi1t)
e1 = -0.5 * e1t
C0 = -20
C1 = -0.1
dt = 0.001
err_ = 0.01

# шаг 1 пройти с шагом 0.1 по всем с и найти близкие орбиты
#C = np.arange(-5.99, -6.04, -0.01)
#look_for_orbit(C)
# шаг 2 пройти с шагом 0.01 по всем с и найти близкие орбиты
#C = np.arange(-7.05, -7.11, -0.01)
#look_for_orbit(C)
# шаг 3 пройти с шагом 0.001 и найти пределы для C сверху и снизу для этого используем
#search(-6.02, -6.04, -1.001, 0.01)
# поменять знаки в функции перед dc
#search(-6.04, -5.99, 0.001, 0.01)
search(-6.03165675, -6.03165655, 0.000000001, 0.01)


# yksi, yeta, yksi1, yeta1 = runge(fksi2, feta2, t0, ksi0, eta0, ksi10, eta10, T0, T, dt)
# #xs = np.arange(T0, T+dt, dt)
# plt.scatter(list(yksi.values()), list(yeta.values()), s=0.1)
# plt.show()

-6.03165675
minr= 2.8110270426007596e-05
minc= -6.03165675
mint= 139.52800000030217
-6.031656749
minr= 2.811016458676798e-05
minc= -6.031656749
mint= 139.52800000030217
-6.031656748
minr= 2.8110063583866085e-05
minc= -6.031656748
mint= 139.52800000030217
-6.0316567469999995
minr= 2.8109966492360063e-05
minc= -6.0316567469999995
mint= 139.52800000030217
-6.0316567459999995
minr= 2.8109862689401247e-05
minc= -6.0316567459999995
mint= 139.52800000030217
-6.031656744999999
minr= 2.8109764927961136e-05
minc= -6.031656744999999
mint= 139.52800000030217
-6.031656743999999
minr= 2.810966771539427e-05
minc= -6.031656743999999
mint= 139.52800000030217
-6.031656742999999
minr= 2.810957096085216e-05
minc= -6.031656742999999
mint= 139.52800000030217
-6.031656741999999
minr= 2.810947415728445e-05
minc= -6.031656741999999
mint= 139.52800000030217
-6.031656740999999
minr= 2.810937935672067e-05
minc= -6.031656740999999
mint= 139.52800000030217
-6.031656739999999
minr= 2.8109283654177666e-05
minc= -6.03